# Recommendation System with Collaborative Filtering

In [1]:
import pypyodbc
import pandas as pd
import numpy as np
from math import *
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from scipy.spatial.distance import cdist

from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist, jaccard
from sqlalchemy import create_engine
import time

## Định nghĩa

**Collaborative Filtering** là một kỹ thuật có thể lọc ra các mục mà người dùng có thể thích trên cơ sở phản ứng của những người dùng tương tự. Cách hoạt động bằng cách tìm kiếm một nhóm lớn người và tìm một nhóm người dùng nhỏ hơn có sở thích tương tự như một người dùng cụ thể, xem xét các mục họ thích và kết hợp chúng để tạo danh sách đề xuất được xếp hạng. 

Đơn vị tính cũng như đề xuất xếp hạng chủ yếu sẽ dựa vào các trọng số tương quan giữa các tập user với nhau. Nhiệm vụ đầu tiên của chúng ta đó chính là tính toán được những trọng số tương quan đó để có thể dự đoán các đề xuất liên quan cho người dùng.

## Các bước thực hiện

- Bước 1: Từ file dữ liệu chứa thông tin các điểm rating của những người dùng đối với các bộ phim khác nhau, ta lập một ma trận giữa người dùng và các bộ phim (user-item matrix)


- Bước 2: Thực hiện tính toán ma trận trên theo phương pháp độ tương tự cosine (cosine similarity) để có ma trận hệ số mức độ tương quan. Từ hệ số tương quan tìm các tập Neighborhood
    - 2.1: Tính khoảng cách euclidean 
    - 2.2: Tính độ tương tự cosine
    - 2.3: Áp dụng K-NN để tìm các tập Neighborhood

- Bước 3: Dự đoán đưa ra các bộ phim phù hợp với người dùng được chỉ định dựa vào tập Neighborhood trên.


- Bước 4: Đánh giá mức độ chính xác 

https://leimao.github.io/blog/Cosine-Similarity-VS-Pearson-Correlation-Coefficient/

#### Bước 1: Tạo ma trận user-item

In [2]:
movie_id = "imdb_id"
author_username = "author"
ratings = "stars"

train_df = pd.read_csv("training_set.csv")
test_df = pd.read_csv("testing_set.csv")

In [3]:
train_df

,date,imdb_id,author,stars
0,2022-07-21,tt0765010,nicktusk-95591,7
1,2022-05-01,tt0765010,peterwixongb,7
2,2022-01-19,tt0765010,kamsitheprince,9
3,2021-09-08,tt0765010,zacharyrivas21,7
4,2021-08-11,tt0765010,SameirAli,7
...,...,...,...,...
365643,2006-12-28,tt0454082,superduperspit,5
365644,2006-12-26,tt0454082,FrightMeter,9
365645,2006-12-25,tt0454082,sackjigler,7
365646,2006-12-25,tt0454082,Nightman85,3


Sử dụng hàm pivot table để tạo ma trận user-item giữa các cột là "stars", "movie_id" và "imdb_id", trong đó "stars" là các điểm rating và "imdb_id" là id định danh người dùng đối với bộ phim "movie_id" tương ứng.

Tuy nhiên, do code này theo hướng tuần tự, thời gian chạy rất lâu nên ta chỉ lấy 1000 user để test. Thông qua code song song ta có thể áp dụng để tính toàn bộ user sau

In [4]:
df_matrix = train_df.pivot_table(index=author_username, columns=movie_id, values=ratings)
df_matrix.reset_index(inplace=True)

# Rút gọn còn 1000 user để test
df_matrix = df_matrix[:1000]
df_matrix

imdb_id,author,tt0000417,tt0010323,tt0013442,tt0015648,tt0015864,tt0017136,tt0020629,tt0021749,tt0021814,...,tt9772374,tt9777666,tt9783600,tt9784456,tt9784798,tt9794630,tt9844522,tt9845564,tt9848626,tt9907782
0,007Waffles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00Yasser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04GreatFlick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0maro0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0w0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Farzad-Doosti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,FatMan-QaTFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,FattyBoomBatty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,FeastMode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Các giá trị NaN có nghĩa là các user này không xem hoặc không để lại đánh giá cho các bộ phim tương ứng.

#### Bước 2: Tính toán Cosine Similarity

Ta sẽ có hai giai đoạn để áp dụng song song đó là tính toán khoảng cách euclidean và độ tương tự cosine. Tuy nhiên trước mắt cần nhận định về thời gian chạy ở code tuần tự.

In [5]:
def calc_cosine_similarity(arr):
    # Caclculate euclidean distance between users, perform parallelization here
    euclidean_distance_matrix = []
    for i in arr:
        user_distance = []
        for j in arr:
            vector_distance = i - j
            euclidean_calc = 0
            for ele in vector_distance:
                if isnan(ele) == False:
                    euclidean_calc += ele*ele
            user_distance.append(sqrt(euclidean_calc))
        euclidean_distance_matrix.append(user_distance)
        
    # Calculate cosine similarity
    euclidean_distance_matrix = np.array(euclidean_distance_matrix).T
    A_sparse = sparse.csr_matrix(euclidean_distance_matrix)
    cosine_similarities = cosine_similarity(A_sparse)

    return cosine_similarities

In [7]:
start = time.time()
test_arr = calc_cosine_similarity(np.array(df_matrix.iloc[:,1:]))
end = time.time()
print(f'Processing time: {end - start} s')

Processing time: 269.8326597213745 s


Như ta có thể thấy, đối với code theo kiểu tuần tự, chỉ với 1000 user mà thời gian tính toán đã khoảng 270s (~=4 phút).

Nếu áp dụng đối với hệ thống trong thực tế, cách làm này sẽ rất tốn thời gian bởi vì đối với các nền tảng streaming ta cần có các kết quả, mức độ phản hồi nhanh nhất có thể để nâng cao trải nghiệm người dùng

In [8]:
user_cosine_similarity = pd.DataFrame(data=test_arr,index=list(df_matrix[author_username]),columns=list(df_matrix[author_username]))
user_cosine_similarity

,007Waffles,00Yasser,04GreatFlick,0maro0,0w0,108YearsOld,109YearsOld,10sion,121mcv,13Funbags,...,FallenEye,Fallenhazel,FallsDownz,FandomFanatic21,Faristuta,Farzad-Doosti,FatMan-QaTFM,FattyBoomBatty,FeastMode,Feel-the-truth
007Waffles,1.000000,0.346984,0.298321,0.525135,0.542194,0.499928,0.539461,0.298269,0.238517,0.218292,...,0.532436,0.096788,0.283915,0.328332,0.495022,0.443352,0.262453,0.548754,0.525805,0.527876
00Yasser,0.346984,1.000000,0.086213,0.274337,0.431617,0.218397,0.155002,0.422531,0.111071,0.255042,...,0.384711,0.188673,0.250300,0.264460,0.361793,0.383920,0.234919,0.377100,0.424072,0.304166
04GreatFlick,0.298321,0.086213,1.000000,0.099290,0.151134,0.087695,0.203956,0.049861,0.076832,0.039455,...,0.136815,0.019153,0.014937,0.066025,0.204296,0.151518,0.051837,0.097185,0.141995,0.167211
0maro0,0.525135,0.274337,0.099290,1.000000,0.508081,0.394769,0.373947,0.328507,0.080608,0.253919,...,0.476631,0.117980,0.237520,0.353755,0.369462,0.329594,0.308251,0.549877,0.499983,0.396691
0w0,0.542194,0.431617,0.151134,0.508081,1.000000,0.436213,0.265203,0.646984,0.147164,0.384080,...,0.752121,0.251302,0.437168,0.472611,0.491372,0.528550,0.410372,0.477065,0.786068,0.488588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Farzad-Doosti,0.443352,0.383920,0.151518,0.329594,0.528550,0.353974,0.228761,0.525080,0.171012,0.282625,...,0.447172,0.224045,0.313303,0.273976,0.342757,1.000000,0.499574,0.337074,0.560237,0.301854
FatMan-QaTFM,0.262453,0.234919,0.051837,0.308251,0.410372,0.298770,0.133574,0.400379,0.040025,0.298456,...,0.340115,0.292653,0.267133,0.226746,0.206890,0.499574,1.000000,0.275361,0.526522,0.156300
FattyBoomBatty,0.548754,0.377100,0.097185,0.549877,0.477065,0.422036,0.312087,0.293309,0.094412,0.258691,...,0.487623,0.089018,0.293692,0.358466,0.392293,0.337074,0.275361,1.000000,0.473930,0.410705
FeastMode,0.525805,0.424072,0.141995,0.499983,0.786068,0.486637,0.293471,0.701135,0.123897,0.477344,...,0.722708,0.312644,0.428257,0.446255,0.480385,0.560237,0.526522,0.473930,1.000000,0.450352


#### Bước 3: Xây dựng tập user Neighborhood bằng k-NN

Xây dựng tập user Neighborhood theo phương pháp k-NN. Đối với từng user sẽ sở hữu các tập Neighborhood riêng được phân chia theo hệ số tương tự ở bảng <b>user_cosine_similarity</b>. Cách thực hiện như sau:

- Loop qua từng user.
- Đối với mỗi user ta tìm tập hợp top các user khác có hệ số tương tự đạt tiêu chuẩn.
- Ta có thể đặt threshold lấy các user có hệ số tương tự cao ở mức 0.95 trở lên hoặc chọn top 100 user cao nhất làm tiêu chuẩn
- Ta sẽ skip các user có độ tương tự = 1 bởi vì chúng là tương tự nhau, không đưa ra được các dữ kiện cần thiết cho mô hình.

In [9]:
# Construct Neighborhood
# Chọn ra top 100 users có độ tương tự cao nhất là các neighbors
K = 100
neighborhood_combine = {}
for user in user_cosine_similarity.columns:
    #neighborhood = list(user_cosine_similarity[user][(user_cosine_similarity[user] > threshold) & (user_cosine_similarity[user] < user_cosine_similarity[user].max())].index)
    neighborhood = list(user_cosine_similarity[user][(user_cosine_similarity[user] < user_cosine_similarity[user].max())].drop_duplicates(keep='first').nlargest(K).index)
    neighborhood_combine[user] = neighborhood

In [10]:
neighborhood_combine

{'007Waffles': ['Draysan-Jennings',
  'Exorcistus',
  'DoNotComeToTheCinemaDepressed',
  'AGood',
  'Eli_Elvis',
  'DongDiggity',
  'Ckn48',
  'ChrisInMiami',
  'Darwinskid',
  'DuskShadow',
  'Erik_Stone',
  'DrTuvok',
  'Crankgorilla',
  'DrProfessor',
  'DanR-Sousa',
  'Abdulxoxo',
  'BrunoRatesTheMovies',
  'Ashraile',
  'DLochner',
  'Behnam-A',
  'Benslie',
  'AlphaBravoCheesecake',
  'Arcadia1701',
  'Birminghamukengland',
  'Benji-80',
  'Boristhemoggy',
  'Entertainmentsparadise',
  'DeadMansTrousers',
  'Anurag-Shetty',
  'Bonnell7',
  'Cinephilia1508',
  'EvanKevelson',
  'DarthWalker-34381',
  'ChrisBeaken',
  'Deadpool2016',
  'Eric266',
  'Eiriksterminator',
  'Celewa',
  'Athanatos173',
  'CtlAltDel',
  'Camerenth',
  'AndersSTHLM',
  '77Lynne',
  'CountJonnie',
  'BoomerFan',
  'Beni_McBeni',
  'Dev1999',
  'DeBBss',
  'Dr_Mark_ODoherty',
  'Alexander_Blanchett',
  'Drawmort',
  'Cineanalyst',
  'Cax019',
  'AliensReservoir',
  'DrCCat',
  'CriticalOfEverything',
  'C0S

Để tăng độ chính xác của k-NN, ta có thể sử dụng thêm k-RNN (k-reciprocal nearest neighbours). Hai đối tượng được gọi là k-RNN của nhau nếu chúng nằm trong số k-NN (tập Neighborhood) của nhau. Như vậy, sau khi đã đạt được tập Neighborhood thì ta sẽ tiến hành kiểm tra tổng quát lại lần nữa dựa theo phương pháp này. Nếu như chúng không nằm trong tập của nhau thì ta sẽ tiến hành xóa nó khỏi tập đó.

In [11]:
# Construct Neighborhood with expanded k-RNN
# Kiểm tra đặc trưng Neighborhood lẫn nhau giữa một user so với các neighbors của chúng
neighborhood_list = []
neighborhood_combine_1 = neighborhood_combine.copy()
for key,value in neighborhood_combine.items():
    for v in value:
        if key in neighborhood_combine[v]:
            continue
        else:
            neighborhood_combine[key].remove(v)

Thực chất, có một vấn đề đối với xử lý này là trong quá trình lược bỏ các user trong tập Neighborhood sẽ có các user mặc dù không nằm trong nhau nhưng lại có hệ số tương tự cao (>0.9) thì ta có thể xem xét không cần lược bỏ các user này đi. Tuy nhiên, ta cần test độ chính xác đối với cách làm này trước khi thực hiện cách thứ hai.

In [12]:
def calc_Neighborhood_kNN(user_arr):
    K = 100
    neighborhood_combine = {}
    for user in user_arr.columns:
        neighborhood = list(user_arr[user][(user_arr[user] < user_arr[user].max())].drop_duplicates(keep='first').nlargest(K).index)
        neighborhood_combine[user] = neighborhood

    for key,value in neighborhood_combine.items():
        for v in value:
            if key in neighborhood_combine[v]:
                continue
            else:
                neighborhood_combine[key].remove(v)

    return neighborhood_combine

In [13]:
start = time.time()
neighborhood_combine = calc_Neighborhood_kNN(user_cosine_similarity)
end = time.time()
print(f'Processing time: {end - start} s')

Processing time: 1.0289978981018066 s


#### Demo recommend

Với bảng hệ số tương quan của các user và tập Neighborhood đã xây dựng, ta sẽ tiến hành đi tìm các bộ phim có thể recommend cho người dùng. Cách làm như sau:

- Chọn một người dùng **x** bất kỳ
- Tìm tập hợp các bộ phim mà người dùng **x** đã xem (tập a)
- Tìm các bộ phim **chung** mà các user trong tập Neighborhood của người dùng **x** này đã đều xem và rating (tập b)
- Lọc bỏ tập a ra khỏi tập b thành tập b' (nếu có)
- Tập b' lúc này chính là những bộ phim với rating mà người dùng **x** chưa xem nhưng được các user trong tập Neighborhood của người dùng **x** này đều đã xem.
- Dựa vào hệ số tương tự đã tính, ta chọn ra các user cao nhất rồi xem đánh giá của họ đối với các bộ phim.
- Chọn ra những bộ phim có rating cao đối với các user này rồi recommend cho người dùng **x**

In [14]:
user_id = '007Waffles'

service_used_by_user_id = df_matrix[df_matrix[author_username] == user_id].dropna(axis=1, how='all')
similar_services = df_matrix[df_matrix[author_username].isin(neighborhood_combine[user_id])].dropna(axis=1, how='all')
similar_services.drop(service_used_by_user_id.columns[1:],axis=1, inplace=True, errors='ignore')
item_score = {}

# Loop through items
for i in similar_services.columns[1:]:
  # Create a variable to store the score
  total = 0
  # Create a variable to store the number of scores
  count = 0
  # Loop through similar users
  for u in neighborhood_combine[user_id]:
    if isnan(similar_services[i][similar_services[author_username] == u].iloc[0]) is False:
    # Score is the sum of user similarity score * rating
      score = user_cosine_similarity[user_id][u] * similar_services[i][similar_services[author_username] == u].values[0]
      total += score
      count +=1

  item_score[i] = total #/ count
# Convert dictionary to pandas dataframe
item_score = pd.DataFrame(item_score.items(), columns=['movie', 'movie_score'])
item_score['count'] = list(similar_services.count().values)[1:]
# Sort the services by score
ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)

# Select top m services
m = 10
print("Top 10 bộ phim được khuyến khích cho user",user_id,"theo mức độ phổ biến")
ranked_item_score[['movie','movie_score']].head(m)

Top 10 bộ phim được khuyến khích cho user 007Waffles theo mức độ phổ biến


,movie,movie_score
559,tt2382320,59.376279
573,tt2527338,56.607708
762,tt6264654,55.820094
650,tt4154796,50.676687
807,tt7131622,49.556791
659,tt4244994,49.455806
648,tt4154664,47.754708
449,tt1477834,46.217473
806,tt7126948,42.463652
897,tt9376612,42.440031


Chỉ số movie_score chính là tổng hệ số tương quan nhân với rating của các người dùng đối xem bộ phim đó. Bởi vì chỉ số được tính tổng nên ta có thể nhận xét rằng những bộ phim này được đề nghị dựa theo mức độ phổ biến, càng nhiều user xem và đánh giá cao trong tập Neighborhood thì chỉ số càng cao.

Bên cạnh đó, nếu ta lấy trung bình chỉ số movie_score này để đánh giá thì nó sẽ biến đổi thành theo "thị hiếu" của những user Neighborhood này. Tức là nếu trung bình càng cao chứng tỏ bộ phim có số điểm rating trung bình càng cao.

In [15]:
user_id = '007Waffles'

service_used_by_user_id = df_matrix[df_matrix[author_username] == user_id].dropna(axis=1, how='all')
similar_services = df_matrix[df_matrix[author_username].isin(neighborhood_combine[user_id])].dropna(axis=1, how='all')
similar_services.drop(service_used_by_user_id.columns[1:],axis=1, inplace=True, errors='ignore')
item_score = {}

# Loop through items
for i in similar_services.columns[1:]:
  # Create a variable to store the score
  total = 0
  # Create a variable to store the number of scores
  count = 0
  # Loop through similar users
  for u in neighborhood_combine[user_id]:
    if isnan(similar_services[i][similar_services[author_username] == u].iloc[0]) is False:
    # Score is the sum of user similarity score * rating
      score = user_cosine_similarity[user_id][u] * similar_services[i][similar_services[author_username] == u].values[0]
      total += score
      count +=1
  item_score[i] = total / count
# Convert dictionary to pandas dataframe
item_score = pd.DataFrame(item_score.items(), columns=['movie', 'movie_score'])
item_score['count'] = list(similar_services.count().values)[1:]
# Sort the services by score
ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)

# Select top m services
m = 10
print("Top 10 bộ phim được khuyến khích cho user",user_id,"theo mức độ đánh giá")
ranked_item_score[['movie','movie_score']].head(m)

Top 10 bộ phim được khuyến khích cho user 007Waffles theo mức độ đánh giá


,movie,movie_score
521,tt1988621,7.032608
388,tt1229238,6.857768
244,tt0437086,6.857768
754,tt6146586,6.857768
601,tt3315342,6.774294
457,tt1506999,6.690820
325,tt1049413,6.424836
223,tt0374546,6.424836
220,tt0372588,6.424836
228,tt0385004,6.424836


Movie_score lúc này còn có thể hiểu nếu user này xem bộ phim này thì sẽ rating bao nhiêu bộ phim đó.

In [16]:
# neighborhood_combine_dic = {i:j for i,j in neighborhood_combine.items() if j != []}
# ser = pd.Series({k: list(v) for k, v in neighborhood_combine_dic.items()}).str.join('|').str.get_dummies()
# ser.replace(0,np.nan,inplace=True)
# out = dict(ser.stack().iteritems())

# neighbor_list = {'user':[],'neighbor':[],'similarity':[]}
# for pair in out:
#     neighbor_list['user'].append(pair[0])
#     neighbor_list['neighbor'].append(pair[1])
#     neighbor_list['similarity'].append(user_cosine_similarity[pair[0]][pair[1]])

In [17]:
def neighborhood_util(neighborhood_combine):
    neighborhood_combine_dic = {i:j for i,j in neighborhood_combine.items() if j != []}
    ser = pd.Series({k: list(v) for k, v in neighborhood_combine_dic.items()}).str.join('|').str.get_dummies()
    ser.replace(0,np.nan,inplace=True)
    out = dict(ser.stack().iteritems())

    neighbor_list = {'user':[],'neighbor':[],'similarity':[]}
    for pair in out:
        neighbor_list['user'].append(pair[0])
        neighbor_list['neighbor'].append(pair[1])
        neighbor_list['similarity'].append(user_cosine_similarity[pair[0]][pair[1]])
    
    return neighbor_list

In [18]:
start = time.time()
neighbor_list = neighborhood_util(neighborhood_combine)
end = time.time()
print(f'Processing time: {end - start} s')

Processing time: 1.263000726699829 s


In [19]:
def calc_rating(selectedUser_neighbors,selectedUser_moviesWatched,final_review,review_matrix,df_selectedUser_neighbors):
    
    test_dict = {}
    for user in selectedUser_neighbors:
        similar_movies = final_review[final_review['author'] == user]['imdb_id'].tolist()
        similar_movies = list(set(similar_movies) - set(selectedUser_moviesWatched))
        for movie in similar_movies:
            get_rating = review_matrix[movie][review_matrix['author'] == user].values[0]
            similarity = df_selectedUser_neighbors[df_selectedUser_neighbors['neighbor'] == user]['similarity'].values[0] * get_rating
            test_dict.setdefault(movie,[]).append(similarity)

    return test_dict


In [20]:
def get_recommend_movies(movie_neighborhood,final_review,user_id,item_limit,mode='popular'):
    check = False
    # Kiểm tra user_id có trong db không, nếu không thì chạy dự đoán. Nếu có thì lấy từ db lên rồi hiển thị
    if check == False:
        review_matrix = final_review.pivot_table(index=author_username, columns=movie_id, values=ratings).reset_index()
        review_matrix = review_matrix[:1000]
        selectedUser_moviesWatched = set(final_review[final_review['author'] == user_id]['imdb_id'].tolist())
        df_selectedUser_neighbors = movie_neighborhood[movie_neighborhood['user'] == user_id].reset_index()
        selectedUser_neighbors = df_selectedUser_neighbors['neighbor'].tolist()

        test_dict = calc_rating(selectedUser_neighbors,selectedUser_moviesWatched,final_review,review_matrix,df_selectedUser_neighbors)

        if mode == 'popular':
            final_dict = sorted(test_dict, key=lambda x:len(test_dict[x]), reverse=True)[:item_limit]
        elif mode == 'favorite':
            final_dict = sorted(test_dict, key=lambda x:sum(test_dict[x]), reverse=True)[:item_limit]
    else:
        print("")
    return final_dict

In [21]:
# Get movie_neighborhood and final_review table from the db
movie_neighborhood = pd.DataFrame(neighbor_list)
final_review = train_df
user_id = '007Waffles'

In [22]:
# Hai mode là popular và favorite
movie_limit = 5
mode = 'popular'
start = time.time()
test_dict = get_recommend_movies(movie_neighborhood,final_review,user_id,movie_limit,mode=mode)
end = time.time()
print(f'Processing time: {end - start} s')

Processing time: 3.118000030517578 s


In [23]:
test_dict

['tt2527338', 'tt4154796', 'tt4154664', 'tt7126948', 'tt2382320']